In [ ]:
from importlib import reload
# For cuda debugging
import os
import json
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product as iter_product

import src, src.debias, src.models, src.ranking, src.datasets, src.data_utils
from src.models import model_loader

if torch.cuda.device_count() > 1:
    use_device_id = int(input(f"Choose cuda index, from [0-{torch.cuda.device_count()-1}]: ").strip())
else: use_device_id = 0
use_device = "cuda:"+str(use_device_id) if torch.cuda.is_available() else "cpu"
if not torch.cuda.is_available():
    input("CUDA isn't available, so using cpu. Please press any key to confirm this isn't an error: \n")
print("Using device", use_device)
torch.cuda.set_device(use_device_id)


In [ ]:
perf_evaluations = ["flickr1k", "cifar100"] # flickr1k, cifar100, cifar10
batch_sz = 256
experiment_results = pd.DataFrame()
db_tokensetting = [(0, "prepend"), (1, "prepend"), (2, "prepend"), (3, "prepend"),
                   (0, "append"), (1, "append"), (2, "append"), (3, "append")]

for n_debias_tokens, db_pos in db_tokensetting:
    with torch.cuda.device(use_device_id):
        model, preprocess, tokenizer, model_alias = src.models.DebiasCLIP.from_cfg(src.Dotdict({
            "CLIP_ARCH": "openai/CLIP/ViT-B/16", "DEVICE": use_device, "num_debias_tokens": n_debias_tokens, "debias_token_init": "zeros", "debias_pos": db_pos
        }))

        for perf_eval in perf_evaluations:
            perf_res = {"debias_pos": db_pos, "n_tokz": n_debias_tokens, "model_name": model_alias, "dataset": perf_eval, "evaluation": perf_eval, "mean": src.debias.run_perf_eval(perf_eval, model, tokenizer, preprocess, use_device)}
            experiment_results = experiment_results.append(pd.DataFrame([perf_res]), ignore_index=True)
    del model, preprocess, tokenizer, model_alias


In [ ]:
display(experiment_results)
